WEBSCRAPING MERCADOLIBRE

In [25]:
import pandas as pd
import matplotlib as plt
from bs4 import BeautifulSoup
import requests

Create a headers

In [171]:
headers = {
    'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'
}

Get de links

In [172]:
productlinks = []
pages = ('', 'Desde_51_', 'Desde_101_', 'Desde_151_')
for x in pages:
    r = requests.get(f'https://listado.mercadolibre.com.co/consolas-videojuegos/consolas/sony/pplaystation-5_{x}NoIndex_True')
    soup = BeautifulSoup(r.content)
    productlist = soup.find_all('div', class_='ui-search-result__content-wrapper')
    for item in productlist:
        for link in item.find_all('a', href=True):
            productlinks.append(link['href'])

Get de information by links

In [173]:
productlist = []
for link in productlinks:
    testlink = link
    r = requests.get(testlink, headers=headers)
    soup = BeautifulSoup(r.content)
    generalinfo = soup.find('div', class_='ui-pdp-header__info')
    price = soup.find('span', class_='andes-money-amount__fraction').text.strip()
    try:
        name = soup.find('h1').text.strip()
    except:
        name = soup.find('div', class_='ui-pdp-header__title-container').text.strip()
    try:
        reviews = generalinfo.find('span', class_='ui-pdp-review__amount').text.strip()
    except:
        reviews = 0
    try:
        rating = generalinfo.find('span', class_='ui-pdp-review__rating').text.strip()
    except:
        rating = 0
    try:
        state = soup.find('span', class_='ui-pdp-subtitle').text.strip()
    except:
        state = 'No Defined'
    try:
        discount = soup.find('span', class_='andes-money-amount__discount').text.strip()
    except:
        discount = 0
    product = {
        'name' : name,
        'price' : price,
        'rating' : rating,
        'reviews' : reviews,
        'state' : state,
        'discount' : discount
    }
    productlist.append(product)

In [175]:
pre_df = pd.DataFrame(productlist)
pre_df.head()

,name,price,rating,reviews,state,discount
0,Consola Playstation 5 Slim Disco Spiderman - P...,2.425.729,0,0,Nuevo | +100 vendidos,29% OFF
1,Consola Sony Playstation 5 Slim Version Lector...,2.284.800,0,0,Nuevo | +100 vendidos,25% OFF
2,Sony PlayStation 5 Slim 1TB Extra DualSense Co...,2.949.800,0,0,Nuevo | +5 vendidos,10% OFF
3,Sony PlayStation 5 825GB Digital Edition color...,2.200.000,0,0,Nuevo | +1000 vendidos,0
4,Sony PlayStation 5 Slim 1TB Call of Duty: Mode...,3.299.900,4.5,(4),Nuevo | +25 vendidos,21% OFF


In [178]:
pre_df.to_csv("new.csv", index=False)

OSError: [Errno 30] Read-only file system: 'new.csv'